In [ ]:

from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
from langchain_groq import ChatGroq
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['LANGSMITH,_API_KEY']=os.getenv('LANGCHAIN_API_KEY')

llm = ChatGroq(model="qwen/qwen3-32b")
response=llm.invoke("Hello")
response.content

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from IPython.display import Image,display

#Graph State
class State(TypedDict):
    topic:str
    characters:str
    settings:str
    premises:str
    story_intro:str

In [ ]:
def generate_characters(state:State):
    """Generate character description"""
    msg = llm.invoke(f"crate two character names and brief traits for a story about {state['topic']}")
    return {"characters":msg.content}


def generate_setting(state:State):
    """Generate a story setting"""
    msg = llm.invoke(f"describe a vivid setting  for a story about {state['topic']}")
    return {"settings":msg.content}


def generate_premise(state:State):
    """Generate a story setting"""
    msg = llm.invoke(f"Write a one sentence plot premises  for a story about {state['topic']}")
    return {"premises":msg.content}

def combine_elements(state:State):
    """Combine characters, setting, and premise into an intro"""
    msg=llm.invoke(
        f"Write a short story introduction using these elements: \n"
        f"Characters:{state['characters']}\n"
        f"Setting:{state['settings']}\n"
        f"Premise:{state['premises']}\n"
    )
    return {"story_intro":msg.content}


In [ ]:
# buil a graph
graph=StateGraph(State)
graph.add_node("character",generate_characters)
graph.add_node("setting",generate_setting)
graph.add_node("premise",generate_premise)
graph.add_node("combine",combine_elements)


In [ ]:
graph.add_edge(START,"character")
graph.add_edge(START,"setting")
graph.add_edge(START,"premise")
graph.add_edge("character","combine")
graph.add_edge("setting","combine")
graph.add_edge("premise","combine")
graph.add_edge("combine",END)

# compile the graph
compiled_graph = graph.compile()

#Visualize thegraph 
graph_image=compiled_graph.get_graph().draw_mermaid_png()
display(Image(graph_image))


In [ ]:
#run the graph
state={"topic":"time travel"}
result=compiled_graph.invoke(state)
result